# Basic Example: Prompt+Model+OutputParser

The most fundamental and commonly used case involves linking a prompt template with a model. To illustrate how this works, let us create a chain that asks for the capital cities of various countries.

**Environtment**

```bash
pip install langsmith langchain langchain_openai langchain_community
```

In [1]:
# Load environment variables

from dotenv import load_dotenv
load_dotenv(override=True, dotenv_path='../.env')

True

## Utilizing Prompt Templates

**`PromptTemplate`**

-   `PromptTemplate` được sử dụng để tạo ra một chuỗi prompt hoàn chỉnh bằng cách kết hợp các biến đầu vào (input variables) từ người dùng.
-   Cách sử dụng:
    -   `template`: Một chuỗi template là một định dạng được định sẵn, nơi dấu ngoặc nhọn '{}' được sử dụng để đại diện cho các biến.
    -   `input_variables`: Tên của các biến cần được chèn vào trong dấu ngoặc nhọn được định nghĩa dưới dạng một danh sách.

**`input_variables`**

-   `input_variables` là một danh sách định nghĩa tên của các biến được sử dụng trong `PromptTemplate`.


In [3]:
from langchain_core.prompts import PromptTemplate

template = "Thành phố của {country} là gì?"
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='Thành phố của {country} là gì?')

In [6]:
prompt = prompt_template.format(country="Việt Nam")
prompt

'Thành phố của Việt Nam là gì?'

In [8]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

## Chain Creation

### LCEL (LangChain Expression Language)

Ở đây, chúng ta sử dụng LCEL (LangChain Expression Language) để kết hợp các thành phần khác nhau thành một chuỗi (chain) duy nhất.

![lcel.png](https://raw.githubusercontent.com/aidino/LangChain-OpenTutorial/683d91eb3fd1028345ee8e5b80660b3b0fcc6b96/01-Basic/assets/02-langchain-expression-language.png)

```
chain = prompt | model | output_parser
```

Đúng vậy, ký hiệu `|` hoạt động tương tự như toán tử pipe trong Unix, liên kết các thành phần khác nhau và truyền đầu ra của thành phần này làm đầu vào cho thành phần tiếp theo.

Trong chuỗi này, đầu vào của người dùng được chuyển đến template prompt, và đầu ra từ template prompt sau đó được chuyển tiếp đến model. Bằng cách xem xét từng thành phần riêng lẻ, bạn có thể hiểu những gì xảy ra ở mỗi bước.


In [10]:
chain = prompt_template | model

In [11]:
chain.invoke("Vietnam")

AIMessage(content='Thành phố của Việt Nam là Hà Nội.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 17, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ab45cd70-5709-4358-8c34-d3e814fb255c-0', usage_metadata={'input_tokens': 17, 'output_tokens': 17, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
chain.invoke("Rusia")

AIMessage(content='Thủ đô của Nga là Moscow.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 18, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-04df5cfa-6d30-4559-9485-31ea13bc679b-0', usage_metadata={'input_tokens': 18, 'output_tokens': 11, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Below is an example of outputting a streaming response:

In [13]:
answer = chain.stream("Vietnam")

for token in answer:
    print(token.content, end="", flush=True)
    
#  flush=True đảm bảo rằng đầu ra được hiển thị ngay lập tức, 
# điều này rất hữu ích trong các tình huống yêu cầu phản hồi theo thời gian thực hoặc hiển thị tiến trình.

Thủ đô của Việt Nam là Hà Nội.

## Output Parser

Một **Output Parser** là một công cụ được thiết kế để chuyển đổi hoặc xử lý các phản hồi từ mô hình AI thành một định dạng cụ thể. Vì đầu ra của mô hình thường được cung cấp dưới dạng văn bản tự do (free-form text), **Output Parser** là cần thiết để chuyển đổi nó thành một định dạng có cấu trúc hoặc trích xuất dữ liệu cần thiết.



In [14]:
from langchain_core.output_parsers import StrOutputParser

output_parser = (
    StrOutputParser()
) # Directly returns the model's response as a string without modification.

prompt_template = PromptTemplate.from_template("Explain about {topic}")
chain = prompt_template | model | output_parser

In [15]:
chain.invoke('Machine Learning')

'Machine learning is a subset of artificial intelligence that involves the development of algorithms and statistical models that enable computers to learn from and make predictions or decisions based on data. \n\nIn machine learning, computers are trained to recognize patterns in data and make decisions or predictions without being explicitly programmed to do so. This is achieved through the use of algorithms that analyze and learn from large amounts of data, known as training data. \n\nThere are several types of machine learning algorithms, including supervised learning, unsupervised learning, and reinforcement learning. In supervised learning, the algorithm is trained on labeled data, where the correct output is provided for each input. In unsupervised learning, the algorithm is trained on unlabeled data and must find patterns or relationships within the data on its own. In reinforcement learning, the algorithm learns through trial and error, receiving feedback in the form of rewards

In [16]:
chain_no_parser = prompt_template | model
chain_no_parser.invoke("Machine Learning")

AIMessage(content='Machine learning is a subset of artificial intelligence that involves the development of algorithms and statistical models that enable computers to learn from and make predictions or decisions based on data. \n\nIn machine learning, computers are trained to recognize patterns in data and make decisions or predictions without being explicitly programmed to do so. This is achieved through the use of algorithms that analyze and learn from large amounts of data, known as training data. \n\nThere are several types of machine learning algorithms, including supervised learning, unsupervised learning, and reinforcement learning. In supervised learning, the algorithm is trained on labeled data, where the correct output is provided for each input. In unsupervised learning, the algorithm is trained on unlabeled data and must find patterns or relationships within the data on its own. Reinforcement learning involves training an algorithm to make decisions based on feedback from i

## Applying and Modifying Templates

In [17]:
template = """
You are a seasoned English teacher with 10 years of experience. Please write an English conversation suitable for the given situation.  
Refer to the [FORMAT] for the structure.

#SITUATION:
{question}

#FORMAT:
- Dialogue in English:
- Explanation of the Dialogue:
"""

prompt = PromptTemplate.from_template(template)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [20]:
response = chain.invoke({"question" : "I want to have my own house"})

In [21]:
print(response)

- Dialogue in English:
Person A: I've been thinking about buying my own house lately.
Person B: That's a great goal to have! Have you started looking at potential properties yet?

- Explanation of the Dialogue:
In this conversation, Person A expresses their desire to own a house. Person B responds positively and asks if Person A has started the process of looking at potential properties. This dialogue shows support and encouragement towards achieving the goal of owning a house.
